In [1]:
#import dependencies
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import pandas as pd
from selenium import webdriver
import json
import pymongo
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, insert, MetaData
import sqlalchemy
import numpy as np
from time import sleep
import requests
from pprint import pprint
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Table 

In [2]:
engine = create_engine('sqlite:///disney_movie_performance.sqlite')
table_drop = engine.execute('DROP TABLE IF EXISTS disney_movie_performance')
conn = engine.connect
meta=MetaData()    
disney_move_performance = Table(
'disney_movie_performance', meta,
Column('index',Integer, primary_key = True),
Column('Year', Integer),
Column('No. 1 Movie', String(255)),
Column('Total Movies Released', String(255)),
Column('Average Production Budget', String(255)),
Column('Combined Worldwide Box Office', String(255)),)

meta.create_all(engine)

In [3]:
engine_2 = create_engine('sqlite:///best_seller.sqlite')
table_drop_2 = engine.execute('DROP TABLE IF EXISTS best_seller')
conn_2 = engine.connect
meta_2=MetaData()   
best_seller = Table(
'best_seller', meta,
Column('index', Integer, primary_key = True),
Column('Overall Best Selling Movie', String(255)),)
meta_2.create_all(engine_2)

In [4]:
#setup splinter
executable_path = {'executable_path':ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.134/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\ravjo\.wdm\drivers\chromedriver\win32\103.0.5060.134]


In [ ]:
[WBD] - Current google-chrome version is 97.0.4692
[WBD] - Get LATEST chromedriver version for 97.0.4692 google-chrome
[WBD] - Driver [C:\Users\ravjo\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache

In [ ]:
[WDM] - Driver has been saved in cache [C:\Users\ravjo\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache

#Walt Disney Movie Index
The table scraped contains data from https://www.the-numbers.com/movies/distributor/Walt-Disney#tab=year. It provides data about movies produced by Disney. It details the top movies for a specific year, a link to stats based on year, movies released per year, average production budget, combined world wide box office revenue, and number one Disney movie for that year.

#Movies by Year
The first table will provide the general information for each year, including best selling movie, movies released, average budget, and total world wide box office revenue.

In [52]:
#connect to website
url = r'https://www.the-numbers.com/movies/distributor/Walt-Disney#tab=year'
browser.visit(url)


In [53]:
html = browser.html
soup = bs(html, 'html.parser')
table = soup.find("table")

# The first tr contains the field names.
headings = [th.get_text().strip() for th in table.find("tr").find_all("th")]

print(headings)

data = []
for row in table.find_all("tr")[1:]:
    table_data = dict(zip(headings, (td.get_text() for td in row.find_all("td"))))
    data.append(table_data)

['Year', 'AnnualStats', 'TotalMovies', 'AverageProductionBudget', 'CombinedWorldwideBox Office', 'No. 1 Movie']


In [54]:
print(data)

[{'Year': '2027', 'AnnualStats': '\xa0', 'TotalMovies': '\xa01', 'AverageProductionBudget': '\xa0', 'CombinedWorldwideBox Office': '\xa0', 'No. 1 Movie': '\xa0'}, {'Year': '2025', 'AnnualStats': '\xa0', 'TotalMovies': '\xa01', 'AverageProductionBudget': '\xa0', 'CombinedWorldwideBox Office': '\xa0', 'No. 1 Movie': '\xa0'}, {'Year': '2024', 'AnnualStats': '\xa0', 'TotalMovies': '\xa08', 'AverageProductionBudget': '\xa0', 'CombinedWorldwideBox Office': '\xa0', 'No. 1 Movie': '\xa0'}, {'Year': '2023', 'AnnualStats': '\xa0', 'TotalMovies': '\xa09', 'AverageProductionBudget': '\xa0', 'CombinedWorldwideBox Office': '\xa0', 'No. 1 Movie': '\xa0'}, {'Year': '2022', 'AnnualStats': 'Annual Stats', 'TotalMovies': '\xa07', 'AverageProductionBudget': '\xa0$191,666,667', 'CombinedWorldwideBox Office': '\xa0$1,119,242,582', 'No. 1 Movie': '\xa0Doctor Strange in the Multiverse of Madness'}, {'Year': '2021', 'AnnualStats': 'Annual Stats', 'TotalMovies': '\xa07', 'AverageProductionBudget': '\xa0$180,000

In [139]:
dfs = pd.DataFrame(data)
dfs


Year   AnnualStats   TotalMovies AverageProductionBudget  \
0    2027                           1                           
1    2025                           1                           
2    2024                           8                           
3    2023                           9                           
4    2022  Annual Stats             7            $191,666,667   
..    ...           ...           ...                     ...   
62   1950                           1              $2,900,000   
63   1946                           1                           
64   1940                           2              $2,284,624   
65   1937                           1              $1,488,000   
66  Total           645   $77,300,792        $104,880,328,203   

   CombinedWorldwideBox Office                                   No. 1 Movie  
0                                                                             
1                                                                             
2                                                                             
3                                                                             
4               $1,119,242,582   Doctor Strange in the Multiverse of Madness  
..                         ...                                           ...  
62                $263,591,415                                    Cinderella  
63                 $65,000,000                             Song of the South  
64                $167,620,832                                     Pinocchio  
65                $184,925,486               Snow White and the Seven Dwarfs  
66           Avengers: Endgame                                           NaN  

[67 rows x 6 columns]

In [140]:
#changes to code to due website updates
#disney_data = pd.merge(total_stats,box_office_stats,how='left')
disney_data = dfs

disney_data.drop(labels=['AnnualStats'],axis=1,inplace=True)



disney_data['Total Movies Released'] = disney_data['TotalMovies']
disney_data['Average Production Budget'] = disney_data['AverageProductionBudget']
disney_data['Combined Worldwide Box Office'] = disney_data['CombinedWorldwideBox Office']
disney_data = disney_data[['Year',
                           'No. 1 Movie',
                           'Total Movies Released',
                           'Average Production Budget', 
                           'Combined Worldwide Box Office']]


disney_data

Year                                   No. 1 Movie Total Movies Released  \
0    2027                                                                   1   
1    2025                                                                   1   
2    2024                                                                   8   
3    2023                                                                   9   
4    2022   Doctor Strange in the Multiverse of Madness                     7   
..    ...                                           ...                   ...   
62   1950                                    Cinderella                     1   
63   1946                             Song of the South                     1   
64   1940                                     Pinocchio                     2   
65   1937               Snow White and the Seven Dwarfs                     1   
66  Total                                           NaN           $77,300,792   

   Average Production Budget Combined Worldwide Box Office  
0                                                           
1                                                           
2                                                           
3                                                           
4               $191,666,667                $1,119,242,582  
..                       ...                           ...  
62                $2,900,000                  $263,591,415  
63                                             $65,000,000  
64                $2,284,624                  $167,620,832  
65                $1,488,000                  $184,925,486  
66          $104,880,328,203             Avengers: Endgame  

[67 rows x 5 columns]

In [141]:
disney_data.columns

Index(['Year', 'No. 1 Movie', 'Total Movies Released',
       'Average Production Budget', 'Combined Worldwide Box Office'],
      dtype='object')

In [142]:
#adds null values to future years
disney_data = disney_data.apply(lambda x: x.str.strip()).replace('', np.nan)

In [143]:
disney_data

Year                                  No. 1 Movie Total Movies Released  \
0    2027                                          NaN                     1   
1    2025                                          NaN                     1   
2    2024                                          NaN                     8   
3    2023                                          NaN                     9   
4    2022  Doctor Strange in the Multiverse of Madness                     7   
..    ...                                          ...                   ...   
62   1950                                   Cinderella                     1   
63   1946                            Song of the South                     1   
64   1940                                    Pinocchio                     2   
65   1937              Snow White and the Seven Dwarfs                     1   
66  Total                                          NaN           $77,300,792   

   Average Production Budget Combined Worldwide Box Office  
0                        NaN                           NaN  
1                        NaN                           NaN  
2                        NaN                           NaN  
3                        NaN                           NaN  
4               $191,666,667                $1,119,242,582  
..                       ...                           ...  
62                $2,900,000                  $263,591,415  
63                       NaN                   $65,000,000  
64                $2,284,624                  $167,620,832  
65                $1,488,000                  $184,925,486  
66          $104,880,328,203             Avengers: Endgame  

[67 rows x 5 columns]

In [144]:
#removes years without data
disney_data=disney_data.dropna()

In [145]:
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', ''))
    return(x)

In [146]:
disney_data['Combined Worldwide Box Office'] = disney_data['Combined Worldwide Box Office'].apply(clean_currency).astype('float')
disney_data['Average Production Budget'] = disney_data['Average Production Budget'].apply(clean_currency).astype('float')

C:\Users\ravjo\AppData\Local\Temp\ipykernel_8596\1272954656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disney_data['Combined Worldwide Box Office'] = disney_data['Combined Worldwide Box Office'].apply(clean_currency).astype('float')
C:\Users\ravjo\AppData\Local\Temp\ipykernel_8596\1272954656.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disney_data['Average Production Budget'] = disney_data['Average Production Budget'].apply(clean_currency).astype('float')


In [136]:
disney_data = disney_data.set_index('Year')

In [147]:
disney_data = disney_data.reset_index(drop=True)

In [148]:
disney_data.head()

Year                                  No. 1 Movie Total Movies Released  \
0  2022  Doctor Strange in the Multiverse of Madness                     7   
1  2021    Shang-Chi and the Legend of the Ten Rings                     7   
2  2020                                       Onward                     1   
3  2019                            Avengers: Endgame                    11   
4  2018                       Avengers: Infinity War                    10   

   Average Production Budget  Combined Worldwide Box Office  
0                191666667.0                   1119242582.0  
1                180000000.0                   1999054531.0  
2                200000000.0                    133317030.0  
3                221888889.0                  11066524160.0  
4                172090000.0                   7023130529.0

In [153]:
disney_data= disney_data[disney_data['No. 1 Movie'].notna()] 


disney_data

disney_data.to_sql('disney_movie_performance', con=engine, if_exists="append")

47

In [154]:
disney_data.columns

Index(['Year', 'No. 1 Movie', 'Total Movies Released',
       'Average Production Budget', 'Combined Worldwide Box Office'],
      dtype='object')

In [155]:
disney_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 0 to 46
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Year                           47 non-null     object 
 1   No. 1 Movie                    47 non-null     object 
 2   Total Movies Released          47 non-null     object 
 3   Average Production Budget      47 non-null     float64
 4   Combined Worldwide Box Office  47 non-null     float64
dtypes: float64(2), object(3)
memory usage: 2.2+ KB


In [156]:
disney_data

Year                                        No. 1 Movie  \
0   2022        Doctor Strange in the Multiverse of Madness   
1   2021          Shang-Chi and the Legend of the Ten Rings   
2   2020                                             Onward   
3   2019                                  Avengers: Endgame   
4   2018                             Avengers: Infinity War   
5   2017                  Star Wars Ep. VIII: The Last Jedi   
6   2016                         Captain America: Civil War   
7   2015               Star Wars Ep. VII: The Force Awakens   
8   2014                            Guardians of the Galaxy   
9   2013                                             Frozen   
10  2012                                       The Avengers   
11  2011        Pirates of the Caribbean: On Stranger Tides   
12  2010                                        Toy Story 3   
13  2009                                                 Up   
14  2008                                             WALL-E   
15  2007           Pirates of the Caribbean: At World’s End   
16  2006         Pirates of the Caribbean: Dead Man’s Chest   
17  2005  The Chronicles of Narnia: The Lion, the Witch ...   
18  2004                                    The Incredibles   
19  2003                                       Finding Nemo   
20  2002                                              Signs   
21  2001                                     Monsters, Inc.   
22  2000                                           Dinosaur   
23  1999                                    The Sixth Sense   
24  1998                                         Armageddon   
25  1997                                           Hercules   
26  1996                                           The Rock   
27  1995                                          Toy Story   
28  1994                                      The Lion King   
29  1993                                      Cool Runnings   
30  1992                                            Aladdin   
31  1991                               Beauty and the Beast   
32  1990                                       Pretty Woman   
33  1989                                 Dead Poets Society   
34  1988                           Who Framed Roger Rabbit?   
35  1987                                   3 Men and a Baby   
36  1986                                    Ruthless People   
37  1985                                 The Black Cauldron   
38  1984                                             Splash   
39  1982                                               Tron   
40  1979                                     The Black Hole   
41  1977                                       The Rescuers   
42  1964                                       Mary Poppins   
43  1954                       20,000 Leagues Under the Sea   
44  1950                                         Cinderella   
45  1940                                          Pinocchio   
46  1937                    Snow White and the Seven Dwarfs   

   Total Movies Released  Average Production Budget  \
0                      7                191666667.0   
1                      7                180000000.0   
2                      1                200000000.0   
3                     11                221888889.0   
4                     10                172090000.0   
5                      8                197428571.0   
6                     15                136153846.0   
7                     11                165055556.0   
8                     12                 89745455.0   
9                     11                125800000.0   
10                    10                113522222.0   
11                    14                 94416667.0   
12                    13                127727273.0   
13                    16                 85090909.0   
14                    13                 75600000.0   
15                    14                 95285714.0   
16                    19                 573

In [157]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)

In [158]:
disney_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 0 to 46
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Year                           47 non-null     object 
 1   No. 1 Movie                    47 non-null     object 
 2   Total Movies Released          47 non-null     object 
 3   Average Production Budget      47 non-null     float64
 4   Combined Worldwide Box Office  47 non-null     float64
dtypes: float64(2), object(3)
memory usage: 2.2+ KB


In [167]:

best_seller = disney_data.sort_values(by='Combined Worldwide Box Office', ascending=False)
best_seller['Combined Worldwide Box Office']=best_seller['Combined Worldwide Box Office'].apply(lambda x: f"${x/1000:.1f}")
best_seller

Year                                        No. 1 Movie  \
3   2019                                  Avengers: Endgame   
6   2016                         Captain America: Civil War   
4   2018                             Avengers: Infinity War   
5   2017                  Star Wars Ep. VIII: The Last Jedi   
7   2015               Star Wars Ep. VII: The Force Awakens   
9   2013                                             Frozen   
12  2010                                        Toy Story 3   
8   2014                            Guardians of the Galaxy   
10  2012                                       The Avengers   
15  2007           Pirates of the Caribbean: At World’s End   
19  2003                                       Finding Nemo   
13  2009                                                 Up   
11  2011        Pirates of the Caribbean: On Stranger Tides   
16  2006         Pirates of the Caribbean: Dead Man’s Chest   
24  1998                                         Armageddon   
23  1999                                    The Sixth Sense   
17  2005  The Chronicles of Narnia: The Lion, the Witch ...   
18  2004                                    The Incredibles   
14  2008                                             WALL-E   
26  1996                                           The Rock   
1   2021          Shang-Chi and the Legend of the Ten Rings   
21  2001                                     Monsters, Inc.   
27  1995                                          Toy Story   
22  2000                                           Dinosaur   
20  2002                                              Signs   
28  1994                                      The Lion King   
25  1997                                           Hercules   
30  1992                                            Aladdin   
0   2022        Doctor Strange in the Multiverse of Madness   
32  1990                                       Pretty Woman   
33  1989                                 Dead Poets Society   
31  1991                               Beauty and the Beast   
29  1993                                      Cool Runnings   
34  1988                           Who Framed Roger Rabbit?   
35  1987                                   3 Men and a Baby   
44  1950                                         Cinderella   
36  1986                                    Ruthless People   
46  1937                    Snow White and the Seven Dwarfs   
45  1940                                          Pinocchio   
41  1977                                       The Rescuers   
2   2020                                             Onward   
42  1964                                       Mary Poppins   
37  1985                                 The Black Cauldron   
38  1984                                             Splash   
39  1982                                               Tron   
40  1979                                     The Black Hole   
43  1954                       20,000 Leagues Under the Sea   

   Total Movies Released  Average Production Budget  \
3                     11                221888889.0   
6                     15                136153846.0   
4                     10                172090000.0   
5                      8                197428571.0   
7                     11                165055556.0   
9                     11                125800000.0   
12                    13                127727273.0   
8                     12                 89745455.0   
10                    10                113522222.0   
15                    14                 95285714.0   
19                    20                 42000000.0   
13                    16                 85090909.0   
11                    14                 94416667.0   
16                    19                 57333333.0   
24                    22                 51111111.0   
23                    22                 51035000.0   
17                    17                 528

In [162]:
best_movie = best_seller.iloc[0,1]
best_movie

'Avengers: Endgame'

In [163]:
best_dict = {'Overall Best Selling Movie': best_movie}
best_seller_df = pd.DataFrame(best_dict, index = [0])

best_seller_df
best_seller_df.to_sql('best_seller', con=engine, if_exists="append")

1

In [164]:
best_seller_df

Overall Best Selling Movie
0          Avengers: Endgame

In [130]:
engine_3 = create_engine('sqlite:///disney_movies_overall.sqlite')
table_drop_3 = engine_3.execute('DROP TABLE IF EXISTS disney_movies_overall')


In [131]:
conn_3 = engine_3.connect()
meta_3=MetaData()
disney_movies_overall = Table(
'disney_movies_overall',meta_3,
Column('index', Integer, primary_key = True),
Column('Release Year Rank', Float),
Column('Movie', String(255)),
Column('ReleaseDate', String(255)),
Column('Genre', String(255)),
Column('MPAARating', String(255)),
Column('Genre', String(255)),
Column('MPAARating', String(255)),  
Column('Year Released', Float),
Column('Gross for Release Year', String(255)),)
meta_3.create_all(engine_3)

In [9]:
years = ['2021',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013',
 '2012',
 '2011',
 '2010',
 '2009',
 '2008',
 '2007',
 '2006',
 '2005',
 '2004',
 '2003',
 '2002',
 '2001',
 '2000',
 '1999',
 '1998',
 '1997',
 '1996',
 '1995',
 '1994',
 '1993',
 '1992',
 '1991',
 '1990',
 '1989',
 '1988',
 '1987',
 '1986',
 '1985',
 '1984',
 '1983',
 '1982',
 '1981',
 '1980',
 '1979',
 '1977',
 '1975',
 '1973',
 '1971',
 '1970',
 '1968',
 '1967',
 '1964',
 '1963',
 '1962',
 '1961',
 '1959',
 '1955',
 '1954',
 '1950',
 '1946',
 '1940',
 '1937']
years

['2021',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013',
 '2012',
 '2011',
 '2010',
 '2009',
 '2008',
 '2007',
 '2006',
 '2005',
 '2004',
 '2003',
 '2002',
 '2001',
 '2000',
 '1999',
 '1998',
 '1997',
 '1996',
 '1995',
 '1994',
 '1993',
 '1992',
 '1991',
 '1990',
 '1989',
 '1988',
 '1987',
 '1986',
 '1985',
 '1984',
 '1983',
 '1982',
 '1981',
 '1980',
 '1979',
 '1977',
 '1975',
 '1973',
 '1971',
 '1970',
 '1968',
 '1967',
 '1964',
 '1963',
 '1962',
 '1961',
 '1959',
 '1955',
 '1954',
 '1950',
 '1946',
 '1940',
 '1937']

In [269]:
#setup splinter
executable_path = {'executable_path':ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)
url_2 = 'https://www.the-numbers.com/market/2022/distributor/Walt-Disney'
browser.visit(url_2)
html = browser.html
soup = bs(html, 'html.parser')



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\ravjo\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [270]:
data = []


for year in years:
    url_2 = 'https://www.the-numbers.com/market/' +str(year) + '/distributor/Walt-Disney'
    pg = requests.get(url_2)
    dd = pd.read_html(pg.text)
    data.append(dd)
#reterieve table data
data

ValueError: No tables found

In [ ]:
len(data)

In [55]:
#test to remove first list item
#hi = data[0]
#hi_1=hi.pop(0)
#hi_1


Rank                         Movie  \
0                           1  Shang-Chi and the Legend of…   
1                           2                   Black Widow   
2                           3                      Eternals   
3                           4                 Jungle Cruise   
4                           5                       Encanto   
5                           6                       Cruella   
6                           7      Raya and the Last Dragon   
7                           8      The Emperor's New Groove   
8                           9           Remember the Titans   
9   Total Gross of All Movies     Total Gross of All Movies   
10         Total Tickets Sold            Total Tickets Sold   

                  ReleaseDate                      Genre  \
0                 Sep 3, 2021                     Action   
1                 Jul 9, 2021                     Action   
2                 Nov 5, 2021                     Action   
3                Jul 30, 2021                  Adventure   
4                Nov 24, 2021                  Adventure   
5                May 28, 2021                     Comedy   
6                 Mar 5, 2021                  Adventure   
7                Dec 15, 2000                  Adventure   
8                Sep 29, 2000                      Drama   
9   Total Gross of All Movies  Total Gross of All Movies   
10         Total Tickets Sold         Total Tickets Sold   

                   MPAARating          2021 Gross  Tickets Sold  
0                       PG-13        $224,543,292    24513460.0  
1                       PG-13        $183,651,655    20049307.0  
2                       PG-13        $164,694,432    17979741.0  
3                       PG-13        $116,987,516    12771562.0  
4                          PG         $91,386,126     9976651.0  
5                       PG-13         $86,103,234     9399916.0  
6                          PG         $54,723,032     5974130.0  
7                           G            $334,000       36462.0  
8                          PG             $65,000        7096.0  
9   Total Gross of All Movies        $922,488,287           NaN  
10         Total Tickets Sold  Total Tickets Sold   100708325.0

In [13]:

#don't run more than once or else removes half of list information
data_1=data
remove = [x.pop(0) for x in data_1]


In [15]:
data_1

[[                         Rank                         Movie  \
  0                           1  Shang-Chi and the Legend of…   
  1                           2                   Black Widow   
  2                           3                      Eternals   
  3                           4                 Jungle Cruise   
  4                           5                       Encanto   
  5                           6                       Cruella   
  6                           7      Raya and the Last Dragon   
  7                           8      The Emperor's New Groove   
  8                           9           Remember the Titans   
  9   Total Gross of All Movies     Total Gross of All Movies   
  10         Total Tickets Sold            Total Tickets Sold   
  
                    ReleaseDate                      Genre  \
  0                 Sep 3, 2021                     Action   
  1                 Jul 9, 2021                     Action   
  2                 Nov 5, 2021

In [80]:
data21=pd.DataFrame(data_1[0][0])
data21['Year Released']= 2021
data21['Gross for Release Year']=data21['2021 Gross']
data21=data21.drop(columns=['2021 Gross','Tickets Sold'])
data21=data21.drop(labels=[9,10],axis=0)
data21

data20=pd.DataFrame(data_1[1][0])
data20['Year Released']=2020
data20['Gross for Release Year']=data20['2020 Gross']
data20=data20.drop(columns=['2020 Gross','Tickets Sold'])
data20=data20.drop(labels=[14,15],axis=0)
data20

data19=pd.DataFrame(data_1[2][0])
data19['Year Released']=2019
data19['Gross for Release Year']=data19['2019 Gross']
data19=data19.drop(columns=['2019 Gross','Tickets Sold'])
data19=data19.drop(labels=[13,14],axis=0)

data19


data18=pd.DataFrame(data_1[3][0])
data18['Year Released']=2018
data18['Gross for Release Year']=data18['2018 Gross']
data18=data18.drop(columns=['2018 Gross','Tickets Sold'])
data18=data18.drop(labels=[13,14],axis=0)


data18


data17=pd.DataFrame(data_1[4][0])
data17['Year Released']=2017
data17['Gross for Release Year']=data17['2017 Gross']
data17=data17.drop(columns=['2017 Gross','Tickets Sold'])
data17=data17.drop(labels=[13,14],axis=0)

data17


data16=pd.DataFrame(data_1[5][0])
data16['Year Released']=2016
data16['Gross for Release Year']=data16['2016 Gross']
data16=data16.drop(columns=['2016 Gross','Tickets Sold'])
data16=data16.drop(labels=[18,19],axis=0)

data16


data15=pd.DataFrame(data_1[6][0])
data15['Year Released']=2015
data15['Gross for Release Year']=data15['2015 Gross']
data15=data15.drop(columns=['2015 Gross','Tickets Sold'])
data15=data15.drop(labels=[15,16],axis=0)

data15

data14=pd.DataFrame(data_1[7][0])
data14['Year Released']=2014
data14['Gross for Release Year']=data14['2014 Gross']
data14=data14.drop(columns=['2014 Gross','Tickets Sold'])
data14=data14.drop(labels=[18,19],axis=0)

data14


data13=pd.DataFrame(data_1[8][0])
data13['Year Released']=2013
data13['Gross for Release Year']=data13['2013 Gross']
data13=data13.drop(columns=['2013 Gross','Tickets Sold'])
data13=data13.drop(labels=[17,18],axis=0)
data13

data12=pd.DataFrame(data_1[9][0])
data12['Year Released']=2012
data12['Gross for Release Year']=data12['2012 Gross']
data12=data12.drop(columns=['2012 Gross','Tickets Sold'])
data12=data12.drop(labels=[18,19],axis=0)
data12

data11=pd.DataFrame(data_1[10][0])
data11['Year Released']=2011
data11['Gross for Release Year']=data11['2011 Gross']
data11=data11.drop(columns=['2011 Gross','Tickets Sold'])
data11=data11.drop(labels=[18,19],axis=0)
data11

data10=pd.DataFrame(data_1[11][0])
data10['Year Released']=2010
data10['Gross for Release Year']=data10['2010 Gross']
data10=data10.drop(columns=['2010 Gross','Tickets Sold'])
data10=data10.drop(labels=[17,18],axis=0)
data10

data09=pd.DataFrame(data_1[12][0])
data09['Year Released']=2009
data09['Gross for Release Year']=data09['2009 Gross']
data09=data09.drop(columns=['2009 Gross','Tickets Sold'])
data09=data09.drop(labels=[23,24],axis=0)
data09

data08=pd.DataFrame(data_1[13][0])
data08['Year Released']=2008
data08['Gross for Release Year']=data08['2008 Gross']
data08=data08.drop(columns=['2008 Gross','Tickets Sold'])
data08=data08.drop(labels=[19,20],axis=0)
data08

data07=pd.DataFrame(data_1[14][0])
data07['Year Released']=2007
data07['Gross for Release Year']=data07['2007 Gross']
data07=data07.drop(columns=['2007 Gross','Tickets Sold'])
data07=data07.drop(labels=[21,22],axis=0)
data07

data06=pd.DataFrame(data_1[15][0])
data06['Year Released']=2006
data06['Gross for Release Year']=data06['2006 Gross']
data06=data06.drop(columns=['2006 Gross','Tickets Sold'])
data06=data06.drop(labels=[25,26],axis=0)
data06


data05=pd.DataFrame(data_1[16][0])
data05['Year Released']=2005
data05['Gross for Release Year']=data05['2005 Gross']
data05=data05.drop(columns=['2005 Gross','Tickets Sold'])
data05=data05.drop(labels=[24,25],axis=0)
data05

data04=pd.DataFrame(data_1[17][0])
data04['Year Released']=2004
data04['Gross for Release Year']=data04['2004 Gross']
data04=data04.drop(columns=['2004 Gross','Tickets Sold'])
data04=data04.drop(labels=[26,27],axis=0)
data04

data03=pd.DataFrame(data_1[18][0])
data03['Year Released']=2003
data03['Gross for Release Year']=data03['2003 Gross']
data03=data03.drop(columns=['2003 Gross','Tickets Sold'])
data03=data03.drop(labels=[31,32],axis=0)
data03

data02=pd.DataFrame(data_1[19][0])
data02['Year Released']=2002
data02['Gross for Release Year']=data02['2002 Gross']
data02=data02.drop(columns=['2002 Gross','Tickets Sold'])
data02=data02.drop(labels=[29,29],axis=0)
data02

data01=pd.DataFrame(data_1[20][0])
data01['Year Released']=2001
data01['Gross for Release Year']=data01['2001 Gross']
data01=data01.drop(columns=['2001 Gross','Tickets Sold'])
data01=data01.drop(labels=[19,20],axis=0)
data01

data00=pd.DataFrame(data_1[21][0])
data00['Year Released']=2000
data00['Gross for Release Year']=data00['2000 Gross']
data00=data00.drop(columns=['2000 Gross','Tickets Sold'])
data00=data00.drop(labels=[28,29],axis=0)
data00

data99=pd.DataFrame(data_1[22][0])
data99['Year Released']=1999
data99['Gross for Release Year']=data99['1999 Gross']
data99=data99.drop(columns=['1999 Gross','Tickets Sold'])
data99=data99.drop(labels=[30,31],axis=0)
data99

data98=pd.DataFrame(data_1[23][0])
data98['Year Released']=1998
data98['Gross for Release Year']=data98['1998 Gross']
data98=data98.drop(columns=['1998 Gross','Tickets Sold'])
data98=data98.drop(labels=[28,29],axis=0)
data98

data97=pd.DataFrame(data_1[24][0])
data97['Year Released']=1997
data97['Gross for Release Year']=data97['1997 Gross']
data97=data97.drop(columns=['1997 Gross','Tickets Sold'])
data97=data97.drop(labels=[34,35],axis=0)
data97

data96=pd.DataFrame(data_1[25][0])
data96['Year Released']=1996
data96['Gross for Release Year']=data96['1996 Gross']
data96=data96.drop(columns=['1996 Gross','Tickets Sold'])
data96=data96.drop(labels=[37,38],axis=0)
data96

data95=pd.DataFrame(data_1[26][0])
data95['Year Released']=1995
data95['Gross for Release Year']=data95['1995 Gross']
data95=data95.drop(columns=["1995 Gross",'Tickets Sold'])
data95=data95.drop(labels=[38,39],axis=0)



In [81]:
list_disney_df=[
data21,
data20,
data19,
data18,
data17,
data16,
data15,
data14,
data13,
data12,
data11,
data10,
data09,
data08,
data07,
data06,
data05,
data04,
data03,
data02,
data01,
data00,
data20,
data99,
data98,
data97,
data96,
data95]
len(list_disney_df)
overall_movie_data=pd.concat(list_disney_df)

In [92]:
overall_movie_data.columns
overall_movie_data=overall_movie_data.rename(columns={"Rank":"Release Year Rank"})
overall_movie_data.columns

Index(['Release Year Rank', 'Movie', 'ReleaseDate', 'Genre', 'MPAARating',
       'Year Released', 'Gross for Release Year'],
      dtype='object')

In [124]:
overall_movie_data.to_sql('disney_movies_overall', con=engine, if_exists="append")

In [110]:
#quit browser
browser.quit()